Base script for comparison existance of output-format in the prompt

In [1]:
# review text

import sys
from pathlib import Path
import pandas as pd

sys.path.append('../../')

import _sample_reviews
import _prompts

from langchain_community.llms import Ollama

# YOUR REVIEW TEXT
review_text = _sample_reviews.sample_11

# YOUR OLLAMA MODEL OF THIS SCRIPT
llm = Ollama(model='gemma:7b-instruct-q4_0', temperature=0.4)

In [2]:
from datetime import datetime
from hashlib import sha224

# create an object with review_ID, review_text and the datetime of creation
def create_review(review_text, review_ID):    
    review_obj = {
        "review_ID": review_ID,
        "review_text": review_text,
        "datetime": datetime.now()
    }

    # create a hash of the review_text for creating unique collection with Chromadb
    hash = sha224(str(review_obj).encode()).hexdigest()
    review_obj['hash'] = hash

    return review_obj

# the temporary review_ID
review_ID = 123
# select a comment to test with
review_obj = create_review(review_text, review_ID)

print(review_obj)
print('\n\n')
print('Len of review_text:', len(review_text.split()))

{'review_ID': 123, 'review_text': 'overall good magic card game. the only issue i can really see is the random algorithem for shuffeling the deck, more times than not you either hit a major mana pocket or no mana at all for half the deck reguardless of deck size. i would also suggest to the devs to allow specific card buying for coins as its really hard to get a single card from an entire set of cards, and yes i would expect the cost to be much higher than for booster packs, adjusted for card value and rareity.', 'datetime': datetime.datetime(2024, 4, 4, 11, 29, 1, 843576), 'hash': '48e6a1efd092f83a4c4b3de9a0c35f1f7caebaca33cce5e00c617f02'}



Len of review_text: 94


In [3]:
# define callbacks for detecting token usage
# copied from ../previous_tests/ollama_get_token_usage_example.ipynb

from langchain_core.callbacks.base import BaseCallbackHandler
from langchain_core.outputs.llm_result import LLMResult
from collections import deque

class TokenUsageCallbackHandler(BaseCallbackHandler):

    def __init__(self, deque: deque = None):
        super().__init__()
        self.deque = deque

    def on_llm_end(self, response: LLMResult, **kwargs) -> None:

        generation = response.generations[0][0]
        gen_info = generation.generation_info

        # get token usage
        # ref: https://github.com/orgs/langfuse/discussions/1179
        token_usage = gen_info.get('prompt_eval_count', 0) + gen_info.get('eval_count', 0)
        # get time costed (local machine)
        # instead of getting total duration, we get the prompt_eval_duration and eval_duration to exclude the load duration (e.g. to load the model to the gpu, etc.)
        time_costed = gen_info.get('prompt_eval_duration', 1e-10) + gen_info.get('eval_duration', 1e-10)     # in ns, a small value to indicate a inf time when it fails


        # create an object to store the token usage and time costed
        token_usage_obj = {
            'token_usage': token_usage,
            'time_costed': time_costed
        }

        # append the object to the deque
        self.deque.append(token_usage_obj)



common_deque = deque()
chain_config = {
    "callbacks": [TokenUsageCallbackHandler(common_deque)],
}

In [4]:
import chromadb

# use docker
# chroma_client = chromadb.HttpClient(host="localhost", port=8000)

In [5]:
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

from langchain.text_splitter import CharacterTextSplitter, TokenTextSplitter, RecursiveCharacterTextSplitter
from transformers import AutoTokenizer
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.vectorstores import Chroma

# also change this to use Mistral Embeddings
embedding_func = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

text_spliter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name='cl100k_base', chunk_size=250, chunk_overlap=40
)

docs = text_spliter.create_documents(
    [review_text], metadatas=[{"source":"review_01"}]
)

temp_db = Chroma.from_documents(
    documents=docs,
    embedding=embedding_func,
    collection_name=review_obj['hash']
)

retriever = temp_db.as_retriever(search_kwargs={"k": 5})

/Users/michaelcheng/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/michaelcheng/miniforge3/envs/fyp-test-tm/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


---

prompting

In [6]:
# global constants
GAME_ASPECTS = ['Gameplay', 'Narrative', 'Accessibility', 'Sound', 'Graphics & Art Design', 'Performance', 'Bug', 'Suggestion', 'Price', 'Overall']

3-3-4

with one-shot output format provided

In [7]:
# 3-3-4
# modified from main branch

from langchain_core.prompts import PromptTemplate, ChatPromptTemplate

token_usage_json_list = []
embedding_usage_info_list = []


for (start, end) in [(0, 3), (3, 6), (6, 10)]:
    aspects = GAME_ASPECTS[start:end]

    my_question = _prompts.QUESTION_TEMPLATE_01 + f"{'is ' if len(aspects) <= 1 else 'are '}" + ': ' + f'{aspects}'
    output_format = _prompts.OUTPUT_FORMAT_TEMPATE.format(
        aspects_list_01=str(aspects)[1:-1].replace('\'', '\"'), output_json_template=str({k: '...' for k in aspects}).replace('\'', '\"')
    )

    relevant_docs = retriever.get_relevant_documents(query=my_question, k=5)


    prompt = PromptTemplate(
        template=_prompts.KEYWORD_TEMPLATE_01,
        input_variables=["aspects", 'output_format', 'summaries'],
    )

    chain = prompt | llm

    _resp = chain.invoke({
        "aspects": aspects,
        "output_format": output_format,
        "summaries": str('\n'.join([d.page_content for d in relevant_docs]))
    }, config=chain_config)

    token_usage_json_list.append(common_deque.pop())

    print(_resp); print('-'*20)

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


{"Gameplay": "The game has a good overall gameplay, but the random algorithm for shuffling the deck can be problematic. It often results in either a major mana pocket or no mana at all for half of the deck.", "Narrative": "NA", "Accessibility": "NA"}

This JSON summarizes the extracted aspects of the game from the reviews.
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


{"Sound": "NA", "Graphics & Art Design": "NA", "Performance": "The random algorithm for shuffling the deck is inconsistent, often resulting in either a major mana pocket or no mana at all for half the deck."}
--------------------
{"Bug": "The random algorithm for shuffling the deck is unreliable, often resulting in unbalanced mana distribution.", "Suggestion": "Allow specific card buying for coins to make it easier to acquire single cards.", "Price": "The cost of booster packs is currently too high, especially considering the rarity of certain cards.", "Overall": "Overall, the game is a good magic card game, but there are some areas that could be improved."}
--------------------


In [8]:
token_usage_json_list

[{'token_usage': 361, 'time_costed': 2288061000},
 {'token_usage': 313, 'time_costed': 1758923000},
 {'token_usage': 355, 'time_costed': 2635835000}]

3-3-4 without output format provided

In [9]:
# without output_format

token_usage_json_list = []
embedding_usage_info_list = []


for (start, end) in [(0, 3), (3, 6), (6, 10)]:
    aspects = GAME_ASPECTS[start:end]

    my_question = _prompts.QUESTION_TEMPLATE_01 + f"{'is ' if len(aspects) <= 1 else 'are '}" + ': ' + f'{aspects}'
    output_format = _prompts.OUTPUT_FORMAT_TEMPATE.format(
        aspects_list_01=str(aspects)[1:-1].replace('\'', '\"'), output_json_template=str({k: '...' for k in aspects}).replace('\'', '\"')
    )

    relevant_docs = retriever.get_relevant_documents(query=my_question, k=5)


    prompt = PromptTemplate(
        template=_prompts.KEYWORD_TEMPLATE_01,
        input_variables=["aspects", 'output_format', 'summaries'],
    )

    chain = prompt | llm

    _resp = chain.invoke({
        "aspects": aspects,
        # "output_format": output_format,
        "output_format": '',
        "summaries": str('\n'.join([d.page_content for d in relevant_docs]))
    }, config=chain_config)

    token_usage_json_list.append(common_deque.pop())

    print(_resp); print('-'*20)

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


**JSON:**

```json
{
  "Gameplay": "The game has a good overall magic card game feel, but the random algorithm for shuffling the deck can be problematic. It often results in either a major mana pocket or no mana at all for half the deck.",
  "Narrative": "NA",
  "Accessibility": "NA"
}
```
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


**JSON:**

```json
{
  "Sound": "NA",
  "Graphics & Art Design": "NA",
  "Performance": "The random algorithm for shuffling the deck is inconsistent, often resulting in either a major mana pocket or no mana at all for half the deck. It would also be helpful to allow specific card buying for coins."
}
```
--------------------
**JSON:**

```json
{
  "Bug": "The random algorithm for shuffling the deck is unreliable, often resulting in either a major mana pocket or no mana at all for half of the deck regardless of deck size.",
  "Suggestion": "The devs should allow specific card buying for coins, as it is currently very difficult to obtain a single card from an entire set of cards.",
  "Price": "The cost of buying single cards would be much higher than booster packs, adjusted for card value and rarity.",
  "Overall": "Overall, the game is good, but there are some issues with the bug and suggestion mentioned above."
}
```
--------------------


In [10]:
token_usage_json_list

[{'token_usage': 278, 'time_costed': 2207553000},
 {'token_usage': 283, 'time_costed': 2242364000},
 {'token_usage': 343, 'time_costed': 3496741000}]

---

2-2-2-2-2 prompting

with output_format provided

In [11]:
token_usage_json_list = []
embedding_usage_info_list = []


for (start, end) in [(0, 2), (2, 4), (4, 6), (6, 8), (8, 10)]:
    aspects = GAME_ASPECTS[start:end]

    my_question = _prompts.QUESTION_TEMPLATE_01 + f"{'is ' if len(aspects) <= 1 else 'are '}" + ': ' + f'{aspects}'
    output_format = _prompts.OUTPUT_FORMAT_TEMPATE.format(
        aspects_list_01=str(aspects)[1:-1].replace('\'', '\"'), output_json_template=str({k: '...' for k in aspects}).replace('\'', '\"')
    )

    relevant_docs = retriever.get_relevant_documents(query=my_question, k=5)


    prompt = PromptTemplate(
        template=_prompts.KEYWORD_TEMPLATE_01,
        input_variables=["aspects", 'output_format', 'summaries'],
    )

    chain = prompt | llm

    _resp = chain.invoke({
        "aspects": aspects,
        "output_format": output_format,
        "summaries": str('\n'.join([d.page_content for d in relevant_docs]))
    }, config=chain_config)

    token_usage_json_list.append(common_deque.pop())

    print(_resp); print('-'*20)

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


{"Gameplay": "The game's shuffle algorithm is inconsistent, often resulting in either a major mana pocket or no mana at all for half the deck.", "Narrative": "NA"}

The text does not provide information about the game's narrative, therefore I cannot extract the requested data.
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


{"Accessibility": "The shuffling algorithm is inconsistent, often resulting in unbalanced mana distribution.", "Sound": "NA"}

The text does not mention sound, therefore I cannot extract the requested aspect.
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


{"Graphics & Art Design": "NA", "Performance": "The only issue with performance is the random algorithm for shuffling the deck."}

The text does not describe the graphics or art design of the game, therefore I cannot extract the requested data.
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


{"Bug": "The random algorithm for shuffling the deck often results in either a major mana pocket or no mana at all for half the deck, regardless of deck size.", "Suggestion": "The devs should allow specific card buying for coins, as it is currently very difficult to obtain a single card from an entire set of cards."}
--------------------
{"Price": "The cost of getting a single card from an entire set of cards is very high, making it difficult to acquire specific cards.", "Overall": "Overall, the game is good, but the random algorithm for shuffling the deck and the high cost of acquiring specific cards are the only issues I can see."}
--------------------


In [12]:
token_usage_json_list

[{'token_usage': 304, 'time_costed': 1919166000},
 {'token_usage': 284, 'time_costed': 1471778000},
 {'token_usage': 305, 'time_costed': 1714605000},
 {'token_usage': 312, 'time_costed': 2143781000},
 {'token_usage': 310, 'time_costed': 2059358000}]

2-2-2-2-2 wihtout output format

In [13]:
token_usage_json_list = []
embedding_usage_info_list = []


for (start, end) in [(0, 2), (2, 4), (4, 6), (6, 8), (8, 10)]:
    aspects = GAME_ASPECTS[start:end]

    my_question = _prompts.QUESTION_TEMPLATE_01 + f"{'is ' if len(aspects) <= 1 else 'are '}" + ': ' + f'{aspects}'
    output_format = _prompts.OUTPUT_FORMAT_TEMPATE.format(
        aspects_list_01=str(aspects)[1:-1].replace('\'', '\"'), output_json_template=str({k: '...' for k in aspects}).replace('\'', '\"')
    )

    relevant_docs = retriever.get_relevant_documents(query=my_question, k=5)


    prompt = PromptTemplate(
        template=_prompts.KEYWORD_TEMPLATE_01,
        input_variables=["aspects", 'output_format', 'summaries'],
    )

    chain = prompt | llm

    _resp = chain.invoke({
        "aspects": aspects,
        "output_format": '',
        "summaries": str('\n'.join([d.page_content for d in relevant_docs]))
    }, config=chain_config)

    token_usage_json_list.append(common_deque.pop())

    print(_resp); print('-'*20)

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


**JSON:**

```json
{
  "Gameplay": "The game has a good overall structure, but the random shuffling algorithm needs improvement. You often either hit a major mana pocket or have no mana at all for half of the deck regardless of deck size.",
  "Narrative": "NA"
}
```

The text does not mention any narrative aspects of the game, therefore I cannot extract the requested data.
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


**JSON:**

```json
{
  "Accessibility": "The game has a high barrier to entry due to its complex rules and mechanics.",
  "Sound": "The game has a good soundscape, but the shuffling algorithm can be inconsistent and result in unbalanced turns."
}
```

**Note:** The text does not describe the game's sound quality or accessibility, therefore I cannot extract the requested data.
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


**JSON:**

```json
{
  "Graphics & Art Design": "NA",
  "Performance": "The only issue with performance is the random algorithm for shuffling the deck, which often results in either a major mana pocket or no mana at all for half the deck regardless of deck size."
}
```

**Explanation:**

The text does not provide information about the game's graphics or art design, therefore I cannot extract the requested data. Hence, the output is "NA" for both aspects.
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


**JSON:**

```json
{
  "Bug": "The random algorithm for shuffling the deck is unreliable, often resulting in major mana pockets or no mana at all for half of the deck regardless of deck size.",
  "Suggestion": "Allow specific card buying for coins. The cost of buying a single card from an entire set of cards is currently very high, and it would be more convenient to be able to purchase individual cards with coins."
}
```

**NA**
--------------------
**JSON:**

```json
{
  "Price": "The cost of buying a single card from an entire set of cards is extremely high, making it difficult to acquire a specific card.",
  "Overall": "Overall, the game is good, but the random algorithm for shuffling the deck and the high cost of buying a single card are the only issues."
}
```
--------------------


In [14]:
token_usage_json_list

[{'token_usage': 286, 'time_costed': 2449819000},
 {'token_usage': 286, 'time_costed': 2436338000},
 {'token_usage': 309, 'time_costed': 2909833000},
 {'token_usage': 299, 'time_costed': 2799309000},
 {'token_usage': 278, 'time_costed': 2286929000}]

---

5-5 with output format provided

In [15]:
token_usage_json_list = []
embedding_usage_info_list = []


for (start, end) in [(0, 5), (5, 10)]:
    aspects = GAME_ASPECTS[start:end]

    my_question = _prompts.QUESTION_TEMPLATE_01 + f"{'is ' if len(aspects) <= 1 else 'are '}" + ': ' + f'{aspects}'
    output_format = _prompts.OUTPUT_FORMAT_TEMPATE.format(
        aspects_list_01=str(aspects)[1:-1].replace('\'', '\"'), output_json_template=str({k: '...' for k in aspects}).replace('\'', '\"')
    )

    relevant_docs = retriever.get_relevant_documents(query=my_question, k=5)


    prompt = PromptTemplate(
        template=_prompts.KEYWORD_TEMPLATE_01,
        input_variables=["aspects", 'output_format', 'summaries'],
    )

    chain = prompt | llm

    _resp = chain.invoke({
        "aspects": aspects,
        "output_format": output_format,
        "summaries": str('\n'.join([d.page_content for d in relevant_docs]))
    }, config=chain_config)

    token_usage_json_list.append(common_deque.pop())

    print(_resp); print('-'*20)

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


{"Gameplay": "The game has a good overall structure, but the random shuffling algorithm needs improvement.", "Narrative": "NA", "Accessibility": "NA", "Sound": "NA", "Graphics & Art Design": "NA"}

The text does not describe the narrative or accessibility of the game, therefore I cannot extract the requested data.
--------------------
{"Performance": "Overall good magic card game", "Bug": "Random algorithem for shuffling the deck", "Suggestion": "Allow specific card buying for coins", "Price": "Really hard to get a single card from an entire set of cards", "Overall": "Overall good magic card game"}
--------------------


In [16]:
token_usage_json_list

[{'token_usage': 356, 'time_costed': 2220377000},
 {'token_usage': 340, 'time_costed': 2083049000}]

5-5 without output format provided

In [17]:
token_usage_json_list = []
embedding_usage_info_list = []


for (start, end) in [(0, 5), (5, 10)]:
    aspects = GAME_ASPECTS[start:end]

    my_question = _prompts.QUESTION_TEMPLATE_01 + f"{'is ' if len(aspects) <= 1 else 'are '}" + ': ' + f'{aspects}'
    output_format = _prompts.OUTPUT_FORMAT_TEMPATE.format(
        aspects_list_01=str(aspects)[1:-1].replace('\'', '\"'), output_json_template=str({k: '...' for k in aspects}).replace('\'', '\"')
    )

    relevant_docs = retriever.get_relevant_documents(query=my_question, k=5)


    prompt = PromptTemplate(
        template=_prompts.KEYWORD_TEMPLATE_01,
        input_variables=["aspects", 'output_format', 'summaries'],
    )

    chain = prompt | llm

    _resp = chain.invoke({
        "aspects": aspects,
        "output_format": '',
        "summaries": str('\n'.join([d.page_content for d in relevant_docs]))
    }, config=chain_config)

    token_usage_json_list.append(common_deque.pop())

    print(_resp); print('-'*20)

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


**JSON:**

```json
{
  "Gameplay": "Overall good magic card game, but the random algorithm for shuffling the deck can be problematic.",
  "Narrative": "NA",
  "Accessibility": "NA",
  "Sound": "NA",
  "Graphics & Art Design": "NA"
}
```

**Note:** The text does not provide information about the game's narrative, accessibility, sound, or graphics & art design, therefore I cannot extract the requested data.
--------------------
**JSON:**

```json
{
  "Performance": "Overall good magic card game",
  "Bug": "Random algorithm for shuffling the deck has issues",
  "Suggestion": "Allow specific card buying for coins",
  "Price": "Cost is high",
  "Overall": "Overall good magic card game"
}
```
--------------------


In [18]:
token_usage_json_list

[{'token_usage': 317, 'time_costed': 2853313000},
 {'token_usage': 282, 'time_costed': 2171431000}]

---

1 aspect per prompt, with output format

In [19]:
for aspect in GAME_ASPECTS:
    my_question = _prompts.QUESTION_TEMPLATE_01 + f"is {aspect}"
    output_format = _prompts.OUTPUT_FORMAT_TEMPATE.format(
        aspects_list_01=str([aspect])[1:-1].replace('\'', '\"'), output_json_template=str({aspect: '...'}).replace('\'', '\"')
    )

    relevant_docs = retriever.get_relevant_documents(query=my_question, k=5)

    prompt = PromptTemplate(
        template=_prompts.KEYWORD_TEMPLATE_01,
        input_variables=["aspects", 'output_format', 'summaries'],
    )

    chain = prompt | llm

    _resp = chain.invoke({
        "aspects": [aspect],
        "output_format": output_format,
        "summaries": str('\n'.join([d.page_content for d in relevant_docs]))
    }, config=chain_config)

    token_usage_json_list.append(common_deque.pop())

    print(_resp); print('-'*20)

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


{"Gameplay": "The gameplay has a major issue with the random algorithm for shuffling the deck, often resulting in either a major mana pocket or no mana at all for half of the deck regardless of deck size. The game also lacks specific card buying for coins, making it difficult to acquire a single card from an entire set of cards."}
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


{"Narrative": "The game's narrative is not present in the text, therefore I cannot extract the requested data."}

NA
--------------------


...

**JSON:**
```
{"Bug": "The shuffling algorithm is unreliable, often resulting in either a major mana pocket or no mana at all for half the deck. The lack of specific card buying for coins makes it difficult to acquire single cards from a set, and the cost of buying booster packs is high."}
```
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


{"Suggestion": "The game needs a more refined shuffling algorithm and the ability to buy specific cards with coins."}

This JSON extracted the "Suggestion" aspect from the reviews and summarizes it in a single paragraph.
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


**JSON:**
{"Price": "The cost of getting a single card from an entire set of cards is very high, making it difficult to acquire specific cards. It would also be nice if the devs allowed for specific card buying for coins."}

This JSON summarizes the aspect of "Price" from the reviews.
--------------------
{"Overall": "The game is overall good, but the random algorithm for shuffling the deck is problematic. The cost of buying specific cards is also an issue."}

This JSON summarizes the extracted aspects of the game from the reviews.
--------------------


In [20]:
token_usage_json_list

[{'token_usage': 317, 'time_costed': 2853313000},
 {'token_usage': 282, 'time_costed': 2171431000},
 {'token_usage': 302, 'time_costed': 2106226000},
 {'token_usage': 261, 'time_costed': 1207891000},
 {'token_usage': 295, 'time_costed': 1937669000},
 {'token_usage': 285, 'time_costed': 1739116000},
 {'token_usage': 275, 'time_costed': 1318208000},
 {'token_usage': 284, 'time_costed': 1699693000},
 {'token_usage': 300, 'time_costed': 2075509000},
 {'token_usage': 277, 'time_costed': 1587496000},
 {'token_usage': 298, 'time_costed': 2018890000},
 {'token_usage': 281, 'time_costed': 1634120000}]

1 aspect per prompt, without output format

In [21]:
for aspect in GAME_ASPECTS:
    my_question = _prompts.QUESTION_TEMPLATE_01 + f"is {aspect}"
    output_format = _prompts.OUTPUT_FORMAT_TEMPATE.format(
        aspects_list_01=str([aspect])[1:-1].replace('\'', '\"'), output_json_template=str({aspect: '...'}).replace('\'', '\"')
    )

    relevant_docs = retriever.get_relevant_documents(query=my_question, k=5)

    prompt = PromptTemplate(
        template=_prompts.KEYWORD_TEMPLATE_01,
        input_variables=["aspects", 'output_format', 'summaries'],
    )

    chain = prompt | llm

    _resp = chain.invoke({
        "aspects": [aspect],
        "output_format": '',
        "summaries": str('\n'.join([d.page_content for d in relevant_docs]))
    }, config=chain_config)

    token_usage_json_list.append(common_deque.pop())

    print(_resp); print('-'*20)

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


**JSON:**

```json
{
  "Gameplay": "The gameplay is overall good, but the random algorithm for shuffling the deck is problematic. It often results in either hitting a major mana pocket or no mana at all for half of the deck, regardless of deck size. The game also lacks specific card buying for coins, making it difficult to obtain a single card from an entire set of cards."
}
```
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


**JSON:**

```json
{
  "Narrative": "The game's narrative is not described in the text, therefore I cannot extract the requested data."
}
```

The text does not describe the game's narrative, therefore I cannot extract the requested data.
--------------------


...

**JSON:**

```json
{
  "Bug": "The shuffling algorithm is unreliable, often resulting in either a major mana pocket or no mana at all for half the deck. It is also difficult to acquire a single card from a set, requiring the purchase of booster packs."
}
```

**Note:** The text does not mention any other aspects of the game besides "Bug", therefore I cannot extract the requested data.
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


**JSON:**

```json
{
  "Suggestion": "The developer should allow specific card buying for coins, and the cost should be much higher than for booster packs, adjusted for card value and rarity."
}
```

**Note:** The text does not specify other aspects of the game, therefore I cannot extract the requested data.
--------------------


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


**JSON:**

```json
{
  "Price": "The cost of acquiring a single card from an entire set of cards is very high, making it difficult to obtain a specific card. The cost would be much higher than for booster packs, adjusted for card value and rarity."
}
```

**Note:** The text does not mention the price of booster packs, therefore I cannot provide an accurate comparison for the cost of acquiring a single card from an entire set of cards versus booster packs.
--------------------
**JSON:**

```json
{
  "Overall": "The game is overall good, but the random algorithm for shuffling the deck is problematic. It often results in either a major mana pocket or no mana at all for half of the deck, regardless of deck size. The game also lacks specific card buying for coins, making it difficult to obtain a single card from an entire set of cards. "
}
```
--------------------
